In [32]:
from bs4 import BeautifulSoup
import requests
import re

response = requests.get("https://en.wikipedia.org/wiki/Kuch_Kuch_Hota_Hai")
doc = BeautifulSoup(response.text)

In [33]:
items = [doc.find('table')]
rows = []

for item in items:
    row = {}

    row['Title_wiki'] = item.select_one('.infobox-above').text.strip()
    row['Director'] = item.find('th', string="Directed by").find_next_sibling('td').get_text(strip=True)
    row['Cast'] = item.find('th', string="Starring").find_next_sibling('td').get_text(separator=", ", strip=True)
    row['Budget'] = item.find('th', string="Budget").find_next_sibling('td').get_text(separator=" ", strip=True).split('[')[0]
    row['Box_Office'] = item.find('th', string="Box office").find_next_sibling('td').get_text(separator=" ", strip=True).split('[')[0]
    row['Music'] = item.find('th', string="Music by").find_next_sibling('td').get_text(strip=True)
    row['Distributor'] = item.find('th', string="Distributed by").find_next_sibling('td').get_text(strip=True)
    row['Release'] = item.find('th', string="Release date").find_next_sibling('td').get_text(strip=True)

    for th in item.find_all('th'):
        if 'Production' in th.get_text() and 'company' in th.get_text():
            row['Production'] = th.find_next_sibling('td').get_text(strip=True)
            break

    # Genre extraction: fix indentation here
    genre = None
    for p in doc.find_all('p'):
        text = p.get_text().lower()
        match = re.search(r'hindi(?:-| )language ([\w\s-]+?) film', text)
        if match:
            genre = match.group(1).strip().title()
            break
    row['Genre'] = genre

    rows.append(row)


In [34]:
import pandas as pd

df = pd.json_normalize(rows)
df

,Title_wiki,Director,Cast,Budget,Box_Office,Music,Distributor,Release,Production,Genre
0,Kuch Kuch Hota Hai,Karan Johar,"Shah Rukh Khan, Kajol, Rani Mukerji, Salman Kh...",₹ 100 million,est. ₹ 1.07 billion,Jatin–Lalit,Yash Raj Films,16 October 1998(1998-10-16)[1],Dharma Productions,Romantic Comedy-Drama


In [24]:
for p in doc.find_all('p'):
    text = p.get_text().lower()
    print("=== PARAGRAPH ===")
    print(text)
    print("=================")
    match = re.search(r'hindi-language ([\w\s-]+?) film', text)
    if match:
        print(">>> MATCH FOUND:", match.group(1))
        genre = match.group(1).strip().title()
        break

row['Genre'] = genre


=== PARAGRAPH ===


=== PARAGRAPH ===
the diplomat is a 2025 indian hindi-language political thriller film directed by shivam nair and written by ritesh shah.[4] the film, starring john abraham and sadia khateeb, is set against the backdrop of india–pakistan relations, exploring themes of diplomacy and the personal conflicts faced by diplomats.[5][6]

>>> MATCH FOUND: political thriller
